# Gausian Naive Bayes On Full Dataset, Applyin PCA

In [1]:
import numpy as np

import pandas as pd

import scipy.stats as s

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
data = pd.read_csv("data.csv")

data.drop([data.columns[0], data.columns[32]], axis =1, inplace = True)

data.head()

diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0         M        17.99         10.38          122.80     1001.0   
1         M        20.57         17.77          132.90     1326.0   
2         M        19.69         21.25          130.00     1203.0   
3         M        11.42         20.38           77.58      386.1   
4         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   symmetry_mean           ...             radius_worst  texture_worst  \
0         0.2419           ...                    25.38          17.33   
1         0.1812           ...                    24.99          23.41   
2         0.2069           ...                    23.57          25.53   
3         0.2597           ...                    14.91          26.50   
4         0.1809           ...                    22.54          16.67   

   perimeter_worst  area_worst  smoothness_worst  compactness_worst  \
0           184.60      2019.0            0.1622             0.6656   
1           158.80      1956.0            0.1238             0.1866   
2           152.50      1709.0            0.1444             0.4245   
3            98.87       567.7            0.2098             0.8663   
4           152.20      1575.0            0.1374             0.2050   

   concavity_worst  concave points_worst  symmetry_worst  \
0           0.7119                0.2654          0.4601   
1           0.2416                0.1860          0.2750   
2           0.4504                0.2430          0.3613   
3           0.6869                0.2575          0.6638   
4           0.4000                0.1625          0.2364   

   fractal_dimension_worst  
0                  0.11890  
1                  0.08902  
2                  0.08758  
3                  0.17300  
4                  0.07678  

[5 rows x 31 columns]

In [3]:
labels = np.array(data['diagnosis']).reshape(data['diagnosis'].shape[0],1)

X = np.array(data.iloc[:,1:])

X.shape

(569, 30)

In [4]:
mu = np.mean(X, axis=0)

mu = mu.reshape(-1,mu.shape[0])

X_dash  = X - mu

sigma_hat = (1/data.shape[0])*np.matmul(X_dash.T,X_dash)

sigma_hat_decomposed = np.linalg.svd(sigma_hat)

Q = sigma_hat_decomposed[0]

lambdaa = sigma_hat_decomposed[1]

Q_tilda = Q[:,0:15]

X_new = np.matmul(X_dash, Q_tilda)

In [5]:
new_data = pd.DataFrame(data=X_new)

new_data['diagnosis'] = labels

In [6]:
training_data_len = int(0.7*new_data.shape[0])

benign_training_data = new_data[new_data['diagnosis'] == 'B'].iloc[0:training_data_len//2]

malignant_training_data = new_data[new_data['diagnosis'] == 'M'].iloc[0:training_data_len//2]

training_data = pd.concat([benign_training_data, malignant_training_data])

In [7]:
cv_data_len = int(0.2 * new_data.shape[0])

begning_remaining_data = new_data[new_data['diagnosis'] == 'B'].iloc[training_data_len//2:]

malignant_remaining_data = new_data[new_data['diagnosis'] == 'M'].iloc[training_data_len//2:]

remaining_data = pd.concat([begning_remaining_data,malignant_remaining_data])

cv_data = remaining_data.iloc[0:cv_data_len]

testing_data = remaining_data.iloc[cv_data_len:]

In [8]:
mu_hat_m = np.array(training_data[training_data['diagnosis'] == 'M'].iloc[:,0:15].mean())

sigma_hat_m = np.array(training_data[training_data['diagnosis'] == 'M'].iloc[:,0:15].cov()) 

np.linalg.det(sigma_hat_m)

malignant_prior = training_data[training_data['diagnosis'] == 'M'].shape[0]/training_data.shape[0]

In [9]:
mu_hat_b = np.array(training_data[training_data['diagnosis'] == 'B'].iloc[:,0:15].mean())

sigma_hat_b = np.array(training_data[training_data['diagnosis'] == 'B'].iloc[:,0:15].cov())

np.linalg.det(sigma_hat_b)

benign_prior = training_data[training_data['diagnosis'] == 'B'].shape[0]/training_data.shape[0]

In [10]:
def cv_data_testing(data):
    
    inputs = np.array(data.iloc[:,0:15])
    
    posterior_m = s.multivariate_normal.pdf(inputs, mu_hat_m, sigma_hat_m) * malignant_prior
    
    posterior_b = s.multivariate_normal.pdf(inputs, mu_hat_b, sigma_hat_b) * benign_prior
    
    boolean_mask = posterior_m > posterior_b
    
    predicted_category = pd.Series(boolean_mask)
    
    predicted_category.replace(to_replace=[False, True], value = ['B', 'M'], inplace=True)
    
    return np.array(predicted_category)

In [11]:
cv_results = cv_data_testing(cv_data)

actual_results = np.array(cv_data['diagnosis'])

boolean_mask = cv_results == actual_results

cv_accuracy = np.count_nonzero(boolean_mask)/boolean_mask.shape[0]

cv_accuracy

0.9292035398230089

In [12]:
testing_results = cv_data_testing(testing_data)

actual_testing_results = np.array(testing_data['diagnosis'])

testing_accuracy = np.count_nonzero(testing_results == actual_testing_results)/actual_testing_results.shape[0]

testing_accuracy

0.9827586206896551